# Muhammad Fazal Hildiansyah - DS2

## Import Library
> Hal pertama yg dilakukan adalah import library yang akan digunakan pada analisa kita

In [2]:
# import pandas
import pandas as pd

# import scaler dari sckit-learn untuk scala point
from sklearn.preprocessing import MinMaxScaler

# import warning agar tidak muncul warning terus di google colab
import warnings
warnings.filterwarnings("ignore")

## Data extraction
> Setelah itu, mulai dari Data Pre-processing, start dari extract data dari source yg akan digunakan

In [3]:
# Baca datasource dari CSV sesuai dengan case
df_raw = pd.read_csv('https://raw.githubusercontent.com/fazalh/fellowship/main/Datasets/nba-seasons-stats.csv')

# Ini merupakan data tambahan, karena nama team di dataset berupa singkatan
nba_abbreviation = pd.read_csv('https://raw.githubusercontent.com/fazalh/fellowship/main/Datasets/nba-abbreviation.csv')

In [4]:
# Merge Data source dengan kepanjangan dari nama Team
df_raw = df_raw.merge(nba_abbreviation, how='left', on='Tm')

In [5]:
# Check data ada berapa column dan row
df_raw.shape

(24691, 54)

## Data Cleansing
> Lalu mulai bersihkan data mengikuti aturan yg berlaku

In [6]:
# Aturan pertama hanya pakai data tahun 2017

df_satu = df_raw[(df_raw['Year'] == 2017)]

df_satu.shape

(595, 54)

In [7]:
# Aturan kedua drop duplicates player

df_dua = df_satu.drop_duplicates(subset=['Player'])

df_dua.shape

(486, 54)

In [8]:
# Check column mana yg paling banyak missing values

df_dua.isnull().sum().sort_values(ascending=False)

blank2        486
blanl         486
3P%            36
FT%            15
2P%             2
FTr             1
TOV%            1
FG%             1
eFG%            1
TS%             1
3PAr            1
2PA             0
Unnamed: 0      0
3PA             0
3P              0
FGA             0
2P              0
FTA             0
FT              0
VORP            0
ORB             0
DRB             0
TRB             0
AST             0
STL             0
BLK             0
TOV             0
PF              0
PTS             0
FG              0
OBPM            0
BPM             0
DRB%            0
Player          0
Pos             0
Age             0
Tm              0
G               0
GS              0
MP              0
PER             0
ORB%            0
TRB%            0
DBPM            0
AST%            0
STL%            0
BLK%            0
USG%            0
OWS             0
DWS             0
WS              0
WS/48           0
Year            0
Team            0
dtype: int64

In [9]:
# Aturan ketiga adalah drop kolom yg banyak missing values

df_tiga = df_dua.drop(['blank2', 'blanl'], axis = 1)

df_tiga.shape

(486, 52)

In [10]:
# Karena kolom 'blank2' dan 'blanl' sudah kita drop,
# selanjutnya cek kolom yg masih ada missing value

df_tiga[['3P%','FT%','2P%','eFG%','TOV%','FG%','FTr','3PAr','TS%']].describe()

,3P%,FT%,2P%,eFG%,TOV%,FG%,FTr,3PAr,TS%
count,450.000000,471.000000,484.000000,485.000000,485.000000,485.000000,485.000000,485.000000,485.000000
mean,0.299342,0.741155,0.482702,0.493454,12.809072,0.441268,0.270179,0.314058,0.526944
std,0.131183,0.139453,0.109347,0.097814,5.332382,0.099094,0.180287,0.209748,0.089771
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.267000,0.675000,0.444750,0.466000,9.600000,0.399000,0.165000,0.156000,0.502000
50%,0.333000,0.766000,0.489500,0.501000,12.400000,0.442000,0.237000,0.324000,0.537000
75%,0.374750,0.832000,0.536000,0.536000,15.400000,0.485000,0.339000,0.455000,0.576000
max,1.000000,1.000000,1.000000,1.000000,43.600000,1.000000,2.000000,1.000000,0.799000


In [11]:
# Jika kita melihat 2 angka dibelakang koma, maka hanya 'FG%' yang symmetric, dan kita bisa impute values dengan nilai rata-rata/mean
df_tiga['FG%'] = df_tiga['FG%'].fillna((df_tiga['FG%'].mean()))

# karena kecenderungan skew maka Kolom lainnya diisi dengan median
df = df_tiga.fillna(df_tiga.median())

In [12]:
# Check ulang apakah masih ada missing values

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486 entries, 24096 to 24690
Data columns (total 52 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  486 non-null    int64  
 1   Year        486 non-null    float64
 2   Player      486 non-null    object 
 3   Pos         486 non-null    object 
 4   Age         486 non-null    float64
 5   Tm          486 non-null    object 
 6   G           486 non-null    float64
 7   GS          486 non-null    float64
 8   MP          486 non-null    float64
 9   PER         486 non-null    float64
 10  TS%         486 non-null    float64
 11  3PAr        486 non-null    float64
 12  FTr         486 non-null    float64
 13  ORB%        486 non-null    float64
 14  DRB%        486 non-null    float64
 15  TRB%        486 non-null    float64
 16  AST%        486 non-null    float64
 17  STL%        486 non-null    float64
 18  BLK%        486 non-null    float64
 19  TOV%        486 non-nul

## Data Analysis

> Setelah data sudah clean, baru memulai analisa sesuai dengan soal

In [13]:
df.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team
24096,24096,2017.0,Alex Abrines,SG,23.0,OKC,68.0,6.0,1055.0,10.1,...,18.0,68.0,86.0,40.0,37.0,8.0,33.0,114.0,406.0,Oklahoma City Thunder
24097,24097,2017.0,Quincy Acy,PF,26.0,TOT,38.0,1.0,558.0,11.8,...,20.0,95.0,115.0,18.0,14.0,15.0,21.0,67.0,222.0,Transferred Players
24100,24100,2017.0,Steven Adams,C,23.0,OKC,80.0,80.0,2389.0,16.5,...,282.0,333.0,615.0,86.0,88.0,78.0,146.0,195.0,905.0,Oklahoma City Thunder
24101,24101,2017.0,Arron Afflalo,SG,31.0,SAC,61.0,45.0,1580.0,9.0,...,9.0,116.0,125.0,78.0,21.0,7.0,42.0,104.0,515.0,Sacramento Kings
24102,24102,2017.0,Alexis Ajinca,C,28.0,NOP,39.0,15.0,584.0,12.9,...,46.0,131.0,177.0,12.0,20.0,22.0,31.0,77.0,207.0,New Orleans Pelicans


### 1. Who is the youngest and oldest player in the NBA in 2017 for each (Tm) ?

In [14]:
# Define younget and oldest player by Team
youngest = df.groupby('Team')['Age', 'Player'].min()
oldest = df.groupby('Team')['Age', 'Player'].max()

# Rename column di tiap dataframe
youngest = youngest.rename(columns={'Age': 'Youngest Age', 'Player': 'Youngest Player'})
oldest = oldest.rename(columns={'Age': 'Oldest Age', 'Player': 'Oldest Player'})

# Merge kedua result diatas
satu = youngest.merge(oldest, how='left', on='Team')
satu

,Youngest Age,Youngest Player,Oldest Age,Oldest Player
Team,,,,
Atlanta Hawks,22.0,DeAndre' Bembry,32.0,Tim Hardaway
Boston Celtics,20.0,Al Horford,31.0,Tyler Zeller
Brooklyn Nets,21.0,Anthony Bennett,36.0,Trevor Booker
Charlotte Hornets,21.0,Aaron Harrison,31.0,Treveon Graham
Chicago Bulls,21.0,Bobby Portis,35.0,Robin Lopez
Cleveland Cavaliers,21.0,Channing Frye,38.0,Tristan Thompson
Dallas Mavericks,21.0,A.J. Hammons,38.0,Wesley Matthews
Denver Nuggets,19.0,Alonzo Gee,36.0,Wilson Chandler
Detroit Pistons,20.0,Andre Drummond,34.0,Tobias Harris


### 2. Which Player has the most minutes played (MP) in each position (Pos) ?

In [15]:
# Define player yang paling sering bermain
dua = df.groupby('Pos')['MP', 'Player'].max()

dua

,MP,Player
Pos,,
C,3030.0,Zaza Pachulia
PF,2803.0,Zach Randolph
PF-C,980.0,Joffrey Lauvergne
PG,2947.0,Yogi Ferrell
SF,3048.0,Wilson Chandler
SG,2796.0,Zach LaVine


In [16]:
# Case Joffrey Lauvergne, dilihat dari data_raw, ternyata lebih sering bermain di Posisi Center
# Saat pindah dari Oklahoma ke Bulls, Joffrey kembali ke posisi semula yaitu Center
# Dan saat research di web NBA, setelah pindah dari Buls ke Spurs, dia juga tetap di Center

df_raw[['Year', 'Team', 'Pos', 'MP']][df_raw['Player'] == 'Joffrey Lauvergne']

,Year,Team,Pos,MP
23202,2015.0,Denver Nuggets,C,268.0
23827,2016.0,Denver Nuggets,C,1041.0
24405,2017.0,Transferred Players,PF-C,980.0
24406,2017.0,Oklahoma City Thunder,PF,739.0
24407,2017.0,Chicago Bulls,C,241.0


In [17]:
# Maka dengan mengubah posisi Joffrey menjadi Center, akan didapatkan hasil seperti ini
df.loc[df['Player'] == 'Joffrey Lauvergne', 'Pos'] = 'C'

df.groupby('Pos')['MP', 'Player'].max()

,MP,Player
Pos,,
C,3030.0,Zaza Pachulia
PF,2803.0,Zach Randolph
PG,2947.0,Yogi Ferrell
SF,3048.0,Wilson Chandler
SG,2796.0,Zach LaVine


### 3. Which team has the highest average total rebound percentage (TRB%), assist percentage (AST%), steal percentage (STL%), and block percentage (BLK%)?


In [18]:
# Define average grouped by Team
tiga = df.groupby('Team')['TRB%', 'AST%', 'STL%', 'BLK%'].mean()
print(tiga)
print('\r')

# print highest TRB
trb = tiga[tiga['TRB%'] == tiga['TRB%'].max()]
print('Highest Average Total Rebound Percentage is ' + trb.index[0])

# print highest AST
ast = tiga[tiga['AST%'] == tiga['AST%'].max()]
print('Highest Average Assist Percentage is ' + ast.index[0])

# print highest STL
stl = tiga[tiga['STL%'] == tiga['STL%'].max()]
print('Highest Average Steal Percentage is ' + stl.index[0])

# print highest BLK
blk = tiga[tiga['BLK%'] == tiga['BLK%'].max()]
print('Highest Average Block Percentage is ' + blk.index[0])

                             TRB%       AST%      STL%      BLK%
Team                                                            
Atlanta Hawks            9.680000  14.120000  1.500000  1.693333
Boston Celtics          10.360000  14.573333  1.366667  1.493333
Brooklyn Nets            9.757143  15.000000  1.457143  1.400000
Charlotte Hornets        9.264286  14.042857  1.235714  1.371429
Chicago Bulls            9.738462  13.061538  1.584615  1.315385
Cleveland Cavaliers     10.142857  11.978571  1.492857  1.885714
Dallas Mavericks        10.400000  14.162500  1.287500  1.643750
Denver Nuggets          10.600000  15.860000  1.880000  0.953333
Detroit Pistons         10.326667  13.200000  1.260000  1.173333
Golden State Warriors   11.426667  13.960000  1.733333  2.740000
Houston Rockets          9.761538  12.823077  1.646154  1.600000
Indiana Pacers          10.153333  12.446667  1.460000  1.540000
Los Angeles Clippers    10.586667  12.033333  1.993333  2.053333
Los Angeles Lakers      1

### 4. Who is the best player in your opinion based on his record stats? note: you can refer to variables point (PTS), assists, rebounds, or anything else. A combination of several variables would be nice.

In [19]:
# Menurut saya, Pemain terbaik di NBA adalah yg memiliki gaya permainan yang baik seperti :
# 1. eFG% - Effective Field Goal Percentage. Sudah mencakup 2 point dan 3 point tembakan.
# 2. TOV% - Turnover Percentage. Kesalahan saat menguasai bola harus rendah agar tidak terjadi turnover, terlihat dari makin rendahnya TOV% makin baik.
# 3. PER - Player Efficiency Rating. Pemain yg paling effisien.
# 4. PTS - Point. Karena di Basket, pemain terbaik adalah pemain yg mencetak score, itu termasuk pertahanan terbaik loh.

# kita buat dataframe untuk soal nomor 4
best_player = df[['Player','eFG%','PER', 'PTS', 'TOV%']].reset_index()

# gunakan MinMaxScaler dengan range dari 0 - 1
scaler = MinMaxScaler(feature_range=(0,1))

# pisahkan number dan category
best_number = best_player[['eFG%','PER', 'PTS', 'TOV%']]
best_name = best_player['Player']

# run scaler dan jadikan dataframe presult
scaler.fit(best_number)
presult = pd.DataFrame(scaler.transform(best_number), columns=['eFG%','PER', 'PTS', 'TOV%'])

# Untuk TOV karena yg makin rendah makin baik, maka kita ganti scalanya dengan 1-value
presult['TOV%'] = 1 - presult['TOV%']

# Jumlahkan semua hasil parameter kolom score
presult['Score'] = presult.sum(axis=1)

# kita gabungkan dengan dataframe best_player dan sort values descending based on Score
pres = pd.concat([best_name, presult], axis=1, sort=False)
pres = pres.sort_values(by=['Score'], ascending=False)
print(pres.head())
print('\r')

print('My best player is ' + pres['Player'][pres['Score'] == pres['Score'].max()])


                 Player   eFG%       PER       PTS      TOV%     Score
457   Russell Westbrook  0.476  0.981670  1.000000  0.635321  3.092991
423       Isaiah Thomas  0.546  0.898167  0.859656  0.754587  3.058410
99        Anthony Davis  0.517  0.918534  0.820563  0.791284  3.047381
432  Karl-Anthony Towns  0.576  0.887984  0.805708  0.740826  3.010517
413      Jarnell Stokes  1.000  1.000000  0.001173  1.000000  3.001173

457    My best player is Russell Westbrook
Name: Player, dtype: object


### 5. Which team has the best average stat record of their players? Note: you can refer to points, assists, rebounds, or anything else. A combination of several variables would be nice

In [20]:
# Menurut saya, Team terbaik di NBA adalah yg memiliki faktor permainan yang baik :
# 1. eFG% - Effective Field Goal Percentage. Sudah mencakup 2 point dan 3 point tembakan.
# 2. TOV% - Turnover Percentage. Kesalahan saat menguasai bola harus rendah agar tidak terjadi turnover, terlihat dari makin rendahnya TOV% makin baik.
# 3. ORB% - Offensive Rebound Percentage. Penguasaan daerah bawah ring lawan.
# 4. DRB% - Defensive Rebound Percentage. Penguasaan daerah bawah ring team.
# 5. FT% - Free Throw Percentage. Seberapa sering melakukan free throw dan dapat point.

# kita buat dataframe untuk soal nomor 5
best_team = df.groupby('Team')['eFG%','TOV%','ORB%','DRB%','FT%'].mean().reset_index()

# gunakan MinMaxScaler dengan range dari 0 - 1
scaler = MinMaxScaler(feature_range=(0,1))

# pisahkan number dan category
team_number = best_team[['eFG%','TOV%','ORB%','DRB%','FT%']]
team_name = best_team['Team']

# run scaler dan jadikan dataframe tresult
scaler.fit(team_number)
tresult = pd.DataFrame(scaler.transform(team_number), columns=['eFG%','TOV%','ORB%','DRB%','FT%'])

# Sama seperti no.4 untuk TOV karena yg makin rendah makin baik, maka kita ganti scalanya dengan 1-value
tresult['TOV%'] = 1 - tresult['TOV%']

# ikuti perhitungan 4 factor dari Dean Oliver
tresult['eFG%'] = (40/100*tresult['eFG%'])
tresult['TOV%'] = (25/100*tresult['TOV%'])
tresult['FT%'] = (15/100*tresult['FT%'])
tresult['RB'] = (20/100*((tresult['ORB%']+tresult['DRB%'])/2))

# assign the score
tresult['Score'] = tresult['eFG%'] + tresult['TOV%'] + tresult['FT%'] + tresult['RB']

# kita gabungkan dengan dataframe team_name dan sort values descending based on Score
tres = pd.concat([team_name, tresult], axis=1, sort=False)
tres = tres.sort_values(by=['Score'], ascending=False)
print(tres.head())
print('\r')

# Hasilnya adalah Boston Celtics, tetapi sayang sekali saat final kalah dengan Golden State Warriors
print('My best team is ' + tres['Team'][tres['Score'] == tres['Score'].max()])



                    Team      eFG%      TOV%      ORB%      DRB%       FT%  \
1         Boston Celtics  0.387467  0.224618  0.553535  0.181310  0.077998   
7         Denver Nuggets  0.400000  0.106995  0.585859  0.221079  0.134595   
30    Washington Wizards  0.305876  0.163453  0.278932  1.000000  0.067958   
12  Los Angeles Clippers  0.238389  0.193086  0.276768  0.329784  0.147995   
10       Houston Rockets  0.357614  0.132939  0.528516  0.041809  0.082774   

          RB     Score  
1   0.073484  0.763568  
7   0.080694  0.722283  
30  0.127893  0.665180  
12  0.060655  0.640126  
10  0.057033  0.630360  

1    My best team is Boston Celtics
Name: Team, dtype: object


## Finish
